## **2025 Data Creator Camp**
같이DATA TEAM: Mission 2

---

## **Google Mount / Data Loading**

구글 드라이브에 저장된 **DCC 데이터셋**을 Google Colab의 Local 환경으로 불러옵니다.

---

### 데이터셋 구성

- **Train Image**: `TS_KS`  
- **Train Label**: `TL_KS_LINE`  
- **Validation Image**: `VS_KS`  
- **Validation Label**: `VL_KS_LINE`

---

In [ ]:
# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 드라이브의 zip 파일을 Colab 로컬 저장소로 복사
!cp /content/drive/MyDrive/DCC2025/Data/Training/01.원천데이터/TS_KS.zip /content/
!cp /content/drive/MyDrive/DCC2025/Data/Training/02.라벨링데이터/TL_KS_LINE.zip /content/
!cp /content/drive/MyDrive/DCC2025/Data/Validation/01.원천데이터/VS_KS.zip /content
!cp /content/drive/MyDrive/DCC2025/Data/Validation/02.라벨링데이터/VL_KS_LINE.zip /content/

# 압축 해제
!unzip /content/TS_KS.zip -d /content/TS_KS
!unzip /content/TL_KS_LINE.zip -d /content/TL_KS_LINE
!unzip /content/VS_KS.zip -d /content/VS_KS
!unzip /content/VL_KS_LINE.zip -d /content/VL_KS_LINE

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: /content/TL_KS_LINE/K3_CHN_20221205052944_82.json  
  inflating: /content/TL_KS_LINE/K3_CHN_20221125053731_47.json  
  inflating: /content/TL_KS_LINE/K3_CHN_20221125053731_56.json  
  inflating: /content/TL_KS_LINE/K3_CHN_20221125053731_64.json  
  inflating: /content/TL_KS_LINE/K3_CHN_20221205052945_0.json  
  inflating: /content/TL_KS_LINE/K3_CHN_20221125053731_73.json  
  inflating: /content/TL_KS_LINE/K3_CHN_20221125053731_53.json  
  inflating: /content/TL_KS_LINE/K3_CHN_20221125053730_17.json  
  inflating: /content/TL_KS_LINE/K3_CHN_20221205052945_74.json  
  inflating: /content/TL_KS_LINE/K3_CHN_20221125053731_67.json  
  inflating: /content/TL_KS_LINE/K3_CHN_20221205052944_62.json  
  inflating: /content/TL_KS_LINE/K3_CHN_20221205052944_30.json  
  inflating: /content/TL_KS_LINE/K3_CHN_20221125053731_68.json  
  inflating: /content/TL_KS_LINE/K3_CHN_20221205052944_12.json  
  inflating: /content/TL_KS_LINE/K3_CHN_20221205052945_

## **Method (Heatmap)**

굴뚝 위치를 강조하여 회귀 모델이 높이에 유효한 영역에 집중하도록 하기 위해 Heatmap을 만드는 과정입니다. <br> 
특히 하나의 RGB에 여러 개의 굴뚝 객체가 존재하는 경우에는 각 객체에 대한 Heatmap을 생성하고, 학습 시 독립적인 입력으로 사용합니다.

---

### INPUT

- **Train Image**: `TS_KS`  
- **Train Label**: `TL_KS_LINE`  
- **Validation Image**: `VS_KS`  
- **Validation Label**: `VL_KS_LINE`

---

### OUTPUT

- **Train Heatmap**: `TS_KS_Grayscale_heat_per_region`  
- **Validation Heatmap**: `VS_KS_Grayscale_heat_per_region`

---

In [ ]:
# -*- coding: utf-8 -*-
"""
per-region 1ch HEAT PNG + OVERLAY PNG 생성 + CSV(manifest) 생성한다.
- HEAT: 0~255의 단일 채널 PNG (가우시안 블러 후 0..1 정규화 → u8 변환) 한다.
- OVERLAY: HEAT에 컬러맵을 입힌 뒤 원본 RGB 위에 알파 합성한 컬러 PNG를 저장한다.
- CSV: heat_png, overlay_png 경로와 region 메타데이터를 기록한다.
"""
import os, json, glob, time, sys, csv
import numpy as np
import cv2

# -------------------------
# 경로 설정
# -------------------------
TRAIN_IMG_DIR  = "./TS_KS"
TRAIN_JSON_DIR = "./TL_KS_LINE"
VAL_IMG_DIR    = "./VS_KS"
VAL_JSON_DIR   = "./VL_KS_LINE"

OUT_DIR_TRAIN  = "./TS_KS_Grayscale_heat_per_region"
OUT_DIR_VAL    = "./VS_KS_Graysacle_heat_per_region"  # 기존 경로 표기 유지한다.
os.makedirs(OUT_DIR_TRAIN, exist_ok=True)
os.makedirs(OUT_DIR_VAL, exist_ok=True)

# -------------------------
# 하이퍼파라미터
# -------------------------
THICKNESS_PX   = 2
GAUSS_KSIZE    = 41
GAUSS_SIGMA    = 8
USE_HEIGHT_AS_WEIGHT_TRAIN = True
USE_HEIGHT_AS_WEIGHT_VAL   = False
HEIGHT_NORM_CLIP = (30.0, 300.0)

# OVERLAY용
COLORMAP        = cv2.COLORMAP_JET
ALPHA_OVERLAY   = 0.45  # heat:alpha, img:(1-alpha)

# -------------------------
# 진행 게이지
# -------------------------
BAR_WIDTH = 28
def print_gauge(prefix, idx, total, ok, warn, t0, last_pct=[-1]):
    import time
    if total <= 0: return
    pct = int(idx * 100 / total)
    if pct == last_pct[0]: return
    last_pct[0] = pct
    filled = int(BAR_WIDTH * pct / 100)
    bar = "█"*filled + "-"*(BAR_WIDTH-filled)
    elapsed = time.time() - t0
    sys.stdout.write(f"\r{prefix} [{bar}] {pct:3d}% ({idx}/{total})  ok:{ok} warn:{warn}  {elapsed:.1f}s")
    sys.stdout.flush()

# -------------------------
# VIA JSON 로드
# -------------------------
def load_via_jsons(json_dir):
    ann = {}
    jf_list = sorted(glob.glob(os.path.join(json_dir, "*.json")))
    if len(jf_list) == 0:
        print(f"[WARN] JSON이 없음: {json_dir}", flush=True)
    for jf in jf_list:
        with open(jf, "r", encoding="utf-8") as f:
            data = json.load(f)
        ann.update(data)
    return ann

# -------------------------
# region 높이 추출/가중치
# -------------------------
def _extract_height(region):
    ra = region.get("region_attributes", {})
    for k in ["chi_height_m", "height_m", "height", "chi_height"]:
        if k in ra:
            try:
                return float(str(ra[k]))
            except:
                pass
    return None

def region_weight(region, use_height_as_weight: bool):
    if not use_height_as_weight:
        return 1.0
    hmin, hmax = HEIGHT_NORM_CLIP
    hval = _extract_height(region)
    if hval is None:
        return 1.0
    hval = max(hmin, min(hmax, hval))
    return 0.5 + 0.5 * (hval - hmin) / (hmax - hmin)

# -------------------------
# HEAT 생성
# -------------------------
def draw_heat_for_region(img_h, img_w, region, use_height_as_weight=False):
    sh = region.get("shape_attributes", {})
    if sh.get("name") != "polyline":
        return None
    xs = sh.get("all_points_x", []); ys = sh.get("all_points_y", [])
    if len(xs) < 2 or len(xs) != len(ys):
        return None

    w = float(region_weight(region, use_height_as_weight))
    heat = np.zeros((img_h, img_w), dtype=np.float32)
    canvas = np.zeros_like(heat, dtype=np.float32)

    pts = np.stack([xs, ys], axis=1).astype(np.int32)
    for i in range(len(pts) - 1):
        p1 = tuple(pts[i]); p2 = tuple(pts[i+1])
        cv2.line(canvas, p1, p2, color=w, thickness=THICKNESS_PX)

    heat += canvas
    k = GAUSS_KSIZE if GAUSS_KSIZE % 2 == 1 else GAUSS_KSIZE + 1
    if k < 3: k = 3
    heat = cv2.GaussianBlur(heat, (k, k), GAUSS_SIGMA)

    if heat.max() > 0:
        heat = heat / (heat.max() + 1e-6)
    return heat  # float32, 0..1

# -------------------------
# SPLIT 처리
# -------------------------
def process_split(img_dir, json_dir, out_dir, split_name="SPLIT"):
    import time
    ann = load_via_jsons(json_dir)
    use_height = USE_HEIGHT_AS_WEIGHT_TRAIN if split_name.upper()=="TRAIN" else USE_HEIGHT_AS_WEIGHT_VAL

    # --- 출력 폴더: HEAT / OVERLAY ---
    dir_heat    = os.path.abspath(os.path.join(out_dir, "HEAT"))
    dir_overlay = os.path.abspath(os.path.join(out_dir, "OVERLAY"))
    os.makedirs(dir_heat, exist_ok=True)
    os.makedirs(dir_overlay, exist_ok=True)

    keys = list(ann.keys())
    total_regions = 0
    for key in keys:
        regs = ann[key].get("regions", [])
        total_regions += sum(1 for r in regs if r.get("shape_attributes", {}).get("name") == "polyline")
    if total_regions == 0:
        print(f"[INFO] {split_name}: polyline region이 없다.", flush=True); return

    csv_path = os.path.join(out_dir, f"{split_name.lower()}_manifest_regions.csv")
    fcsv = open(csv_path, "w", newline="", encoding="utf-8")
    wr = csv.writer(fcsv)
    wr.writerow([
        "heat_png","overlay_png",
        "img_filename","region_index","chi_id",
        "height_m","all_points_x","all_points_y","used_height_weight"
    ])

    cnt_done = cnt_warn = processed = 0
    t0 = time.time()

    for key in keys:
        item = ann[key]
        filename = item.get("filename"); regions = item.get("regions", [])
        if not filename:
            missed = sum(1 for r in regions if r.get("shape_attributes", {}).get("name") == "polyline")
            cnt_warn += missed; processed += missed
            print_gauge(f"[{split_name}]", processed, total_regions, cnt_done, cnt_warn, t0)
            continue

        img_path = os.path.join(img_dir, filename)
        if not os.path.isfile(img_path):
            base_noext = os.path.splitext(filename)[0]
            alt = glob.glob(os.path.join(img_dir, base_noext) + ".*")
            if len(alt) > 0: img_path = alt[0]
        img = cv2.imread(img_path, cv2.IMREAD_COLOR) if os.path.isfile(img_path) else None
        if img is None:
            missed = sum(1 for r in regions if r.get("shape_attributes", {}).get("name") == "polyline")
            cnt_warn += missed; processed += missed
            print_gauge(f"[{split_name}]", processed, total_regions, cnt_done, cnt_warn, t0)
            continue

        H, W = img.shape[:2]
        reg_idx = -1
        for r in regions:
            if r.get("shape_attributes", {}).get("name") != "polyline": continue
            reg_idx += 1

            heat01 = draw_heat_for_region(H, W, r, use_height_as_weight=use_height)
            if heat01 is None:
                cnt_warn += 1; processed += 1
                print_gauge(f"[{split_name}]", processed, total_regions, cnt_done, cnt_warn, t0)
                continue

            # --- HEAT 저장 (1채널 u8) ---
            heat1_u8 = np.clip(heat01 * 255.0, 0, 255).astype(np.uint8)

            # --- OVERLAY 생성 (컬러맵 + 알파합성) ---
            heat_color_bgr = cv2.applyColorMap(heat1_u8, COLORMAP)               # (H,W,3) u8
            overlay_bgr    = cv2.addWeighted(heat_color_bgr, ALPHA_OVERLAY,
                                             img, 1.0 - ALPHA_OVERLAY, 0)

            base  = os.path.splitext(os.path.basename(img_path))[0]
            chi_id = str(r.get("region_attributes", {}).get("chi_id","")).strip() or "x"
            tag   = f"{base}_reg{reg_idx:03d}_chi{chi_id}"

            out_heat1 = os.path.abspath(os.path.join(dir_heat,    f"{tag}_heat1.png"))
            out_over  = os.path.abspath(os.path.join(dir_overlay, f"{tag}_overlay.png"))

            ok1 = cv2.imwrite(out_heat1, heat1_u8)
            ok2 = cv2.imwrite(out_over,  overlay_bgr)

            if not (ok1 and ok2):
                cnt_warn += 1
            else:
                cnt_done += 1
                xs = r.get("shape_attributes", {}).get("all_points_x", []) or []
                ys = r.get("shape_attributes", {}).get("all_points_y", []) or []
                xs_str = ";".join(map(str, xs))
                ys_str = ";".join(map(str, ys))
                h_m = _extract_height(r)
                wr.writerow([
                    out_heat1, out_over,
                    filename, reg_idx, chi_id,
                    "" if h_m is None else f"{h_m:.6f}",
                    xs_str, ys_str, int(bool(use_height))
                ])

            processed += 1
            print_gauge(f"[{split_name}]", processed, total_regions, cnt_done, cnt_warn, t0)

    sys.stdout.write("\n")
    elapsed = time.time() - t0
    print(f"[DONE] {split_name}: {json_dir} -> {out_dir}, regions {total_regions}, 성공 {cnt_done}, 경고 {cnt_warn}, {elapsed:.1f}s")
    fcsv.close()
    print(f"[CSV] {csv_path}")

# -------------------------
# main
# -------------------------
def main():
    process_split(TRAIN_IMG_DIR, TRAIN_JSON_DIR, OUT_DIR_TRAIN, split_name="TRAIN")
    process_split(VAL_IMG_DIR,   VAL_JSON_DIR,   OUT_DIR_VAL,   split_name="VAL")

if __name__ == "__main__":
    main()


[TRAIN] [████████████████████████████] 100% (10590/10590)  ok:10590 warn:0  289.0s
[DONE] TRAIN: ./TL_KS_LINE -> ./TS_KS_Grayscale_heat_per_region, regions 10590, 성공 10590, 경고 0, 289.0s
[CSV] ./TS_KS_Grayscale_heat_per_region/train_manifest_regions.csv
[VAL] [████████████████████████████] 100% (1323/1323)  ok:1323 warn:0  36.0s
[DONE] VAL: ./VL_KS_LINE -> ./VS_KS_Graysacle_heat_per_region, regions 1323, 성공 1323, 경고 0, 36.0s
[CSV] ./VS_KS_Graysacle_heat_per_region/val_manifest_regions.csv


## **Data Oversampling**

굴뚝 높이 데이터셋 불균형으로 인해 희소한 높이 데이터에서 RMSE 오차가 큰 것을 확인하였습니다. <br>
이를 해결하기 위해 높이 120m 이상 데이터 비율을 높이는 Data Oversampling을 진행하여 데이터 분포 균형을 맞춥니다. <br>

이 과정은 Validation에서는 진행하지 않고, 오직 Training에서만 진행합니다. 

### Setting
- **Oversampling Mode**: mutual (HEIGHT_TH를 경계로 낮은 영역과 높은 영역을 분리해서 Data Oversampling 진행)
- **HEIGHT_TH** = 120 (여기서 설정한 높이를 기준으로 Data Oversampling을 진행)
- **UP_LEFT** = 0.00 (HEIGHT_TH보다 낮은 영역에 해당하는 비율)
- **UP_RIGHT** = 0.40 (HEIGHT_TH보다 높은 영역에 해당하는 비율)

---

### INPUT

- **Train Heatmap**: `TS_KS_Grayscale_heat_per_region`  

---

### OUTPUT

- **Train csv**: `TS_KS_Grayscale_heat_per_region/train_manifest_regions_uniform.csv`  

---

In [ ]:
# -*- coding: utf-8 -*-
"""
Mission 2: 입력 CSV(UNION or MANIFEST)를 안전하게 읽어
          ▶ '기존 manifest(Val과 동일) 스키마'로 변환 + '오버샘플링' 적용하여 새 파일로 저장

입력(변경하지 않음):
  /content/TS_KS_Grayscale_heat_per_region/train_manifest_regions.csv
  - 가능 스키마 A (UNION):
      patch_path, height_m, img_path, chi_id, chi_height_m, points_x, points_y, source_json, region_index
  - 가능 스키마 B (MANIFEST; Val과 동일):
      heat_png, overlay_png, img_filename, region_index, chi_id, height_m, all_points_x, all_points_y, used_height_weight

출력(새로 생성; Val 스키마/순서 고정):
  /content/TS_KS_Grayscale_heat_per_region/train_manifest_regions_uniform.csv
  - columns:
      heat_png, overlay_png, img_filename, region_index, chi_id, height_m,
      all_points_x, all_points_y, used_height_weight

부가 산출물(분포 확인):
  - /content/TS_KS_Grayscale_heat_per_region/hist_before_union2manifest.png
  - /content/TS_KS_Grayscale_heat_per_region/hist_after_union2manifest.png
  - /content/TS_KS_Grayscale_heat_per_region/hist_before_union2manifest.csv
  - /content/TS_KS_Grayscale_heat_per_region/hist_after_union2manifest.csv
"""

import os, csv, math, shutil
import numpy as np
import matplotlib.pyplot as plt

# ======================= 경로/설정 =======================
INPUT_CSV   = "/content/TS_KS_Grayscale_heat_per_region/train_manifest_regions.csv"      # 원본(유지)
OUT_DIR     = "/content/TS_KS_Grayscale_heat_per_region"
OUT_CSV     = os.path.join(OUT_DIR, "train_manifest_regions_uniform.csv")                # 새로 생성

# 히스토그램/오버샘플 파라미터
HEIGHT_TH = 120.0       # 왼/오른쪽 경계
BINS      = 35
SEED      = 42

# 오버샘플링 정책
#  - "global_uniform": 모든 bin을 (최빈 bin * factor)까지 올림
#  - "mutual"       : 왼쪽(<TH)/오른쪽(>=TH)을 서로의 최빈값 비율로 올림
OVERSAMPLE_MODE  = "mutual"   # "global_uniform" | "mutual"
UP_FACTOR_GLOBAL = 1.00       # global_uniform 전용
UP_LEFT          = 0.00       # mutual 왼쪽 타깃 = 오른쪽 최대 * UP_LEFT
UP_RIGHT         = 0.40       # mutual 오른쪽 타깃 = 왼쪽 최대 * UP_RIGHT (예: 왼쪽이 많을 때 오른쪽 소수 보강)

VAL_SCHEMA_HEADER = [
    "heat_png","overlay_png","img_filename","region_index","chi_id",
    "height_m","all_points_x","all_points_y","used_height_weight"
]

# ======================= 유틸 함수 =======================
def _ensure_dir(p):
    d = os.path.dirname(p) if os.path.splitext(p)[1] else p
    if d and not os.path.exists(d):
        os.makedirs(d, exist_ok=True)

def _derive_overlay_from_heat(heat_png_path: str) -> str:
    """
    규칙:
      /.../HEAT/XXXX_heat1.png  ->  /.../OVERLAY/XXXX_overlay.png
    (파일명이 다르면 HEAT->OVERLAY 디렉토리 치환만 적용)
    """
    if not heat_png_path:
        return ""
    out = heat_png_path.replace("/HEAT/", "/OVERLAY/")
    base = os.path.basename(out)
    if "heat1" in base:
        base = base.replace("heat1", "overlay")
        out = os.path.join(os.path.dirname(out), base)
    return out

def _detect_delimiter(filepath, default=","):
    """csv.Sniffer 로 구분자 자동 탐지 (콤마/탭 우선). 실패 시 헤더 1줄을 보고 탭 여부 판단."""
    with open(filepath, "r", encoding="utf-8") as f:
        sample = f.read(4096)
    try:
        sniffer = csv.Sniffer()
        dialect = sniffer.sniff(sample, delimiters=[",", "\t", ";", "|"])
        return dialect.delimiter
    except Exception:
        first = sample.splitlines()[0] if sample else ""
        if "\t" in first:
            return "\t"
        return default

def _clean_key(k: str) -> str:
    # 앞 BOM 제거 + 공백 제거 + 소문자
    return (k or "").strip().lstrip("\ufeff").lower()

def save_hist(values, bins, outfile_png, outfile_csv, th=120.0, title="Distribution", bin_edges=None):
    values = np.asarray(values, dtype=float)
    if bin_edges is None:
        counts, be = np.histogram(values, bins=bins)
        bin_edges = be
    else:
        counts, _ = np.histogram(values, bins=bin_edges)

    _ensure_dir(outfile_csv); _ensure_dir(outfile_png)
    with open(outfile_csv, "w", newline="", encoding="utf-8") as f:
        w = csv.writer(f); w.writerow(["bin_left","bin_right","count"])
        for i in range(len(counts)):
            w.writerow([bin_edges[i], bin_edges[i+1], int(counts[i])])

    plt.figure(figsize=(8,4.5))
    plt.hist(values, bins=bin_edges, edgecolor="black", alpha=0.7)
    plt.axvline(th, linestyle="--", color="red", label=f"{int(th)} m")
    plt.title(title); plt.xlabel("Height (m)"); plt.ylabel("Frequency")
    plt.legend(); plt.tight_layout(); plt.savefig(outfile_png, dpi=200); plt.close()
    print(f"[저장] {outfile_png}, {outfile_csv}")

# ======================= 1) 입력 CSV 로드 (스키마 자동 판별) =======================
def load_rows_as_manifest_val_schema(input_csv_path):
    """
    - 입력: UNION 스키마 또는 MANIFEST(Val) 스키마
    - 출력: MANIFEST(Val) 스키마 dict 리스트
    """
    if not os.path.exists(input_csv_path):
        raise FileNotFoundError(f"입력 CSV를 찾을 수 없다: {input_csv_path}")

    delim = _detect_delimiter(input_csv_path)
    print(f"[INFO] Detected delimiter: {repr(delim)} for {input_csv_path}")

    rows = []
    with open(input_csv_path, "r", encoding="utf-8") as f:
        rd_raw = csv.reader(f, delimiter=delim)
        try:
            headers = next(rd_raw)
        except StopIteration:
            raise RuntimeError("입력 CSV가 비어 있습니다.")
        headers = [_clean_key(h) for h in headers]

        # DictReader 수동 구성
        rd = (dict(zip(headers, row)) for row in rd_raw)

        hdr_set = set(headers)
        has_union    = {"patch_path","height_m","img_path"}.issubset(hdr_set)
        has_manifest = {"heat_png","height_m","img_filename"}.issubset(hdr_set)

        if not has_union and not has_manifest:
            raise RuntimeError(
                f"알 수 없는 스키마입니다. 헤더={headers}\n"
                f"- UNION 필요한 헤더 예: patch_path, img_path, height_m\n"
                f"- MANIFEST 필요한 헤더 예: heat_png, img_filename, height_m"
            )

        n_total, n_ok = 0, 0
        for r in rd:
            n_total += 1
            r = { _clean_key(k): (v.strip() if isinstance(v, str) else v) for k,v in r.items() }

            try:
                if has_union:
                    heat_png = r.get("patch_path","")
                    img_path = r.get("img_path","")
                    height_m = float(r.get("height_m",""))

                    row_manf = {
                        "heat_png": heat_png,
                        "overlay_png": _derive_overlay_from_heat(heat_png),
                        "img_filename": os.path.basename(img_path) if img_path else "",
                        "region_index": r.get("region_index",""),
                        "chi_id": r.get("chi_id",""),
                        "height_m": height_m,
                        "all_points_x": r.get("points_x",""),
                        "all_points_y": r.get("points_y",""),
                        "used_height_weight": r.get("used_height_weight","1") or "1",
                    }
                else:
                    heat_png = r.get("heat_png","")
                    img_fn   = r.get("img_filename","")
                    height_m = float(r.get("height_m",""))

                    row_manf = {
                        "heat_png": heat_png,
                        "overlay_png": r.get("overlay_png","") or _derive_overlay_from_heat(heat_png),
                        "img_filename": img_fn,
                        "region_index": r.get("region_index",""),
                        "chi_id": r.get("chi_id",""),
                        "height_m": height_m,
                        "all_points_x": r.get("all_points_x","") or r.get("points_x",""),
                        "all_points_y": r.get("all_points_y","") or r.get("points_y",""),
                        "used_height_weight": r.get("used_height_weight","1") or "1",
                    }
                rows.append(row_manf)
                n_ok += 1
            except Exception:
                # height_m 파싱 실패 등은 스킵
                continue

    if not rows:
        raise RuntimeError("입력 CSV에서 유효한 행을 하나도 만들지 못했습니다.")
    print(f"[로드] rows={n_ok}/{n_total} (ok/total)")

    # sanity check: 열 값에 경로 대신 숫자만 들어가는 문제 방지
    bad = sum(1 for r in rows if (not r["heat_png"]) or r["heat_png"].isdigit())
    if bad > 0:
        print(f"[경고] heat_png가 비었거나 숫자인 행 {bad}개 발견. 구분자/헤더를 다시 확인하세요.")

    return rows

# ======================= 2) 오버샘플링 =======================
def oversample_manifest_rows(rows, bins=BINS, th=HEIGHT_TH,
                             mode=OVERSAMPLE_MODE,
                             up_factor_global=UP_FACTOR_GLOBAL,
                             up_left=UP_LEFT, up_right=UP_RIGHT,
                             seed=SEED):
    rng = np.random.default_rng(seed)
    heights = np.array([r["height_m"] for r in rows], dtype=float)

    hmin, hmax = float(heights.min()), float(heights.max())
    bin_edges = np.linspace(hmin, hmax, bins + 1)

    bin_ids = np.digitize(heights, bin_edges[:-1], right=False) - 1
    bin_ids = np.clip(bin_ids, 0, bins - 1)

    indices_per_bin = [[] for _ in range(bins)]
    for idx, b in enumerate(bin_ids):
        indices_per_bin[b].append(idx)

    counts_before = np.array([len(idxs) for idxs in indices_per_bin], dtype=int)

    left_mask  = (bin_edges[:-1] < th)
    right_mask = ~left_mask

    targets = np.array(counts_before, dtype=int)

    if mode == "global_uniform":
        up_factor_global = max(1.0, float(up_factor_global))
        global_max = int(counts_before.max()) if counts_before.size else 0
        target_all = int(math.ceil(global_max * up_factor_global))
        targets[:] = target_all
        print(f"[정책: global_uniform] global_max={global_max}, target/bin={target_all}")

    elif mode == "mutual":
        up_left  = float(up_left); up_right = float(up_right)
        max_left  = int(counts_before[left_mask].max())  if left_mask.any()  else 0
        max_right = int(counts_before[right_mask].max()) if right_mask.any() else 0
        target_left  = int(math.ceil(max_right * up_left))
        target_right = int(math.ceil(max_left  * up_right))
        targets[left_mask]  = target_left
        targets[right_mask] = target_right
        print(f"[정책: mutual] max_left={max_left}, max_right={max_right}, "
              f"target_left={target_left}, target_right={target_right}")
    else:
        raise ValueError("OVERSAMPLE_MODE는 'global_uniform' 또는 'mutual'")

    selected_idx = list(range(len(rows)))  # 원본 유지
    skipped_zero_bins = []
    for b in range(bins):
        c = counts_before[b]; t = targets[b]
        if c == 0:
            if t > 0:
                skipped_zero_bins.append(b)
            continue
        need = max(0, t - c)
        if need > 0:
            dup = rng.choice(indices_per_bin[b], size=need, replace=True)
            selected_idx.extend(dup.tolist())

    oversampled = [rows[i].copy() for i in selected_idx]
    for r in oversampled:
        r["used_height_weight"] = "1"

    counts_after = np.histogram([r["height_m"] for r in oversampled], bins=bin_edges)[0]

    if skipped_zero_bins:
        print(f"[경고] 원본 0개 bin {len(skipped_zero_bins)}개 건너뜀: {skipped_zero_bins[:10]} ...")
    print(f"[오버샘플] 원본={len(rows)} → 이후={len(oversampled)} (x{len(oversampled)/len(rows):.2f})")

    return oversampled, bin_edges, counts_before, counts_after

# ======================= 3) 저장(Val 스키마, 새 파일) =======================
def save_manifest_csv_val_schema(rows, out_csv_path):
    _ensure_dir(out_csv_path)
    with open(out_csv_path, "w", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        w.writerow(VAL_SCHEMA_HEADER)  # 컬럼/순서 '검증 manifest'와 동일
        for r in rows:
            w.writerow([
                r.get("heat_png",""),
                r.get("overlay_png",""),
                r.get("img_filename",""),
                r.get("region_index",""),
                r.get("chi_id",""),
                f"{float(r.get('height_m',0.0)):.6f}",
                r.get("all_points_x",""),
                r.get("all_points_y",""),
                r.get("used_height_weight","1"),
            ])
    print(f"[저장] 새 파일 생성(Val 스키마): {out_csv_path}")

# ======================= 메인 =======================
if __name__ == "__main__":
    _ensure_dir(OUT_DIR)

    # 1) 입력 로드(스키마 자동 판별 → Val 스키마 dict 리스트로 통일)
    rows_manf = load_rows_as_manifest_val_schema(INPUT_CSV)
    heights_before = [r["height_m"] for r in rows_manf]

    # 2) 분포 저장(이전)
    save_hist(heights_before, BINS,
              os.path.join(OUT_DIR, "hist_before_union2manifest.png"),
              os.path.join(OUT_DIR, "hist_before_union2manifest.csv"),
              th=HEIGHT_TH, title="Original height distribution (TRAIN source)")

    # 3) 오버샘플링
    oversampled_rows, bin_edges, c_before, c_after = oversample_manifest_rows(
        rows_manf, bins=BINS, th=HEIGHT_TH, mode=OVERSAMPLE_MODE,
        up_factor_global=UP_FACTOR_GLOBAL, up_left=UP_LEFT, up_right=UP_RIGHT, seed=SEED
    )
    heights_after = [r["height_m"] for r in oversampled_rows]

    # 4) 분포 저장(이후)
    title = "Global-uniform oversampling" if OVERSAMPLE_MODE == "global_uniform" else "Mutual oversampling"
    save_hist(heights_after, BINS,
              os.path.join(OUT_DIR, "hist_after_union2manifest.png"),
              os.path.join(OUT_DIR, "hist_after_union2manifest.csv"),
              th=HEIGHT_TH, title=title, bin_edges=bin_edges)

    # 5) 'Val 스키마'로 동일 순서의 새 CSV 저장 (원본은 변경하지 않음)
    save_manifest_csv_val_schema(oversampled_rows, OUT_CSV)

    # 6) 요약
    b_low  = sum(h < HEIGHT_TH for h in heights_before)
    b_high = sum(h >= HEIGHT_TH for h in heights_before)
    a_low  = sum(h < HEIGHT_TH for h in heights_after)
    a_high = sum(h >= HEIGHT_TH for h in heights_after)
    print(f"[요약] 원본 : <{HEIGHT_TH}m={b_low}개, >={HEIGHT_TH}m={b_high}개, 총={len(heights_before)}개")
    print(f"[요약] 이후 : <{HEIGHT_TH}m={a_low}개, >={HEIGHT_TH}m={a_high}개, 총={len(heights_after)}개")
    print(f"[완료] 출력 CSV: {OUT_CSV}")


[INFO] Detected delimiter: ',' for /content/TS_KS_Grayscale_heat_per_region/train_manifest_regions.csv
[로드] rows=10590/10590 (ok/total)
[저장] /content/TS_KS_Grayscale_heat_per_region/hist_before_union2manifest.png, /content/TS_KS_Grayscale_heat_per_region/hist_before_union2manifest.csv
[정책: mutual] max_left=946, max_right=638, target_left=0, target_right=379
[오버샘플] 원본=10590 → 이후=17250 (x1.63)
[저장] /content/TS_KS_Grayscale_heat_per_region/hist_after_union2manifest.png, /content/TS_KS_Grayscale_heat_per_region/hist_after_union2manifest.csv
[저장] 새 파일 생성(Val 스키마): /content/TS_KS_Grayscale_heat_per_region/train_manifest_regions_uniform.csv
[요약] 원본 : <120.0m=6990개, >=120.0m=3600개, 총=10590개
[요약] 이후 : <120.0m=6990개, >=120.0m=10260개, 총=17250개
[완료] 출력 CSV: /content/TS_KS_Grayscale_heat_per_region/train_manifest_regions_uniform.csv


## **Model Training & Ensemble Learning**

RGB, Heatmap을 각각 독립된 백본 (ResNet)에 넣고 마지막에 결합하여 높이 추정을 진행하는 Late Fusion 방식으로 구현하였습니다. <br>
또한 굴뚝 높이 데이터의 불균형과 큰 높이 분포 편차로 인해 단일 모델의 예측이 불안정한 상황이기 때문에 더 안정적이고 우수한 성능을 위해 Ensemble Learning 기법을 적용하였습니다. <br>

Ensemble Learning은 5개의 모델 결과를 결합합니다. 최종 RMSE 결과는 **4.4457**입니다.

---

### Data Augmentation

- **Horizontal Flip**: 0.2
- **Vertical Flip**: 0.2
- **Brightness**: 0.3 (±30% 범위 내에서 밝기를 무작위로 변화)

---

### INPUT
- **Train Image**: `TS_KS`
- **Train csv**: `TS_KS_Grayscale_heat_per_region/train_manifest_regions_uniform.csv`

- **Validation Image**: `VS_KS`
- **Validation csv**: `VS_KS_Grayscale_heat_per_region/val_manifest_regions_uniform.csv`
---

### OUTPUT
- **Ensemble Learning Result**: `latefusion_ensemble`  

---

In [ ]:
# ===== Cell 1: 공통 모듈 작성 및 설정 =====
# 코랩 로컬에 공통 모듈을 저장한다.
import os, json, csv, glob, random, importlib, math
import numpy as np
import cv2
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import functional as TF
from torchvision.transforms import InterpolationMode, RandomResizedCrop
import matplotlib.pyplot as plt

# =========================
# 경로/출력 설정
# =========================
TRAIN_CSV_MANIFEST = "/content/TS_KS_Grayscale_heat_per_region/train_manifest_regions_uniform.csv"
VAL_CSV_MANIFEST   = "/content/VS_KS_Graysacle_heat_per_region/val_manifest_regions.csv"
TRAIN_RGB_ROOT     = "/content/TS_KS"
VAL_RGB_ROOT       = "/content/VS_KS"

# 결과 저장
OUT_BASE = "/content/drive/MyDrive/DCC2025/Mission_2/latefusion_ensemble"
os.makedirs(OUT_BASE, exist_ok=True)

# =========================
# 앙상블/하이퍼파라미터
# =========================
ENSEMBLE_SIZE = 5
BASE_SEED     = 42

IMG_SIZE     = 512
BATCH_SIZE   = 64
EPOCHS       = 50
LR           = 3e-4
WEIGHT_DECAY = 1e-4
NUM_WORKERS  = 4
DEVICE       = "cuda" if torch.cuda.is_available() else "cpu"

# 인코더 스위치
RGB_BACKEND, RGB_ARCH, RGB_WEIGHTS   = "torchvision", "resnet18", "IMAGENET1K_V1"
HEAT_BACKEND, HEAT_ARCH, HEAT_WEIGHTS = "torchvision", "resnet18", "IMAGENET1K_V1"

# 증강 설정
AUG_ENABLE              = True
AUG_RANDOM_RESIZED_CROP = False
AUG_RRC_SCALE           = (1.00, 1.00)
AUG_RRC_RATIO           = (1.00, 1.10)
AUG_HFLIP_P             = 0.20
AUG_VFLIP_P             = 0.20
AUG_ROTATE_DEG          = 0.00
AUG_TRANSPOSE90_P       = 0.00
# 색상 증강(RGB만)
AUG_COLORJIT_P          = 0.50
AUG_BRIGHTNESS          = 0.30
AUG_CONTRAST            = 0.00
AUG_SATURATION          = 0.00
AUG_HUE                 = 0.00
AUG_GAMMA_P             = 0.00
AUG_GAMMA_RANGE         = (0.8, 1.25)
AUG_GRAY_P              = 0.00
AUG_GBLUR_P             = 0.00
AUG_GBLUR_K             = 3
AUG_GBLUR_SIGMA         = (0.1, 1.0)

# =========================================================
# 유틸/모델/데이터셋 구현부
# =========================================================
def set_global_seed(seed:int):
    import torch.backends.cudnn as cudnn
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
    cudnn.benchmark = False; cudnn.deterministic = True
    torch.backends.cuda.matmul.allow_tf32 = False
    cudnn.allow_tf32 = False
    torch.use_deterministic_algorithms(True, warn_only=True)

def _safe_getattr(module, name):
    return getattr(module, name) if hasattr(module, name) else None

def _enum_name_tv(arch):
    return {"resnet18":"ResNet18_Weights","resnet34":"ResNet34_Weights","resnet50":"ResNet50_Weights"}.get(arch)

def _list_enum_members(enum_cls):
    return [k for k in dir(enum_cls) if k.isupper() and not k.startswith("_")]

def load_resnet(backend="torchvision", arch="resnet18", weights=None, remove_fc=True, verbose=True):
    backend = backend.lower(); arch = arch.lower()
    if backend == "torchvision":
        tv = importlib.import_module("torchvision.models")
        constructor = _safe_getattr(tv, arch)
        if constructor is None:
            raise ValueError(f"[torchvision] 지원하지 않는 arch: {arch}")
        enum_name = _enum_name_tv(arch)
        weights_enum = _safe_getattr(tv, enum_name) if enum_name else None
        weights_obj = None
        if isinstance(weights, str) and weights_enum is not None:
            cand = _safe_getattr(weights_enum, weights)
            if cand is None and verbose:
                print(f"[WARN] torchvision {enum_name}.{weights} 없음. 후보:", _list_enum_members(weights_enum))
            weights_obj = cand
        m = constructor(weights=weights_obj)
    else:
        raise ValueError("backend는 torchvision만 사용한다")
    feat_dim = None
    if remove_fc and hasattr(m, "fc") and isinstance(m.fc, nn.Linear):
        feat_dim = m.fc.in_features
        m.fc = nn.Identity()
    else:
        feat_dim = getattr(m, "fc").in_features if hasattr(m, "fc") else 512
    if verbose:
        print(f"[load] backend={backend} arch={arch} weights={weights} -> feat_dim={feat_dim}")
    return m, feat_dim

def _float_or_none(s):
    if s is None: return None
    s = str(s).strip()
    if s == "": return None
    try: return float(s)
    except: return None

def _collapse_dup(path_str):
    parts = []
    for p in os.path.normpath(path_str).split(os.sep):
        if not parts or parts[-1] != p: parts.append(p)
    return (os.sep + os.path.join(*parts)) if path_str.startswith(os.sep) else os.path.join(*parts)

def _normalize_path(raw_p, csv_dir):
    if not raw_p: return None
    p = raw_p.strip()
    if os.path.isabs(p):
        return p if os.path.isfile(p) else None
    p = os.path.normpath(os.path.join(csv_dir, p))
    if os.path.isfile(p): return p
    p2 = _collapse_dup(p)
    if os.path.isfile(p2): return p2
    base = os.path.basename(p)
    hits = glob.glob(os.path.join(csv_dir, "**", base), recursive=True)
    if hits: return hits[0]
    parent = os.path.abspath(os.path.join(csv_dir, ".."))
    hits = glob.glob(os.path.join(parent, "**", base), recursive=True)
    return hits[0] if hits else None

def _find_rgb_path_from_filename(rgb_root, img_filename):
    if not img_filename: return None
    if os.path.isabs(img_filename) and os.path.isfile(img_filename): return img_filename
    base = os.path.basename(img_filename)
    cand = os.path.join(rgb_root, base)
    if os.path.isfile(cand): return cand
    stem, _ = os.path.splitext(base)
    for ext in [".jpg",".JPG",".png",".PNG",".jpeg",".JPEG"]:
        q = os.path.join(rgb_root, stem + ext)
        if os.path.isfile(q): return q
    return None

def load_items_from_manifest_csv(csv_path, rgb_root, filter_used_flag=True):
    items = []; miss_heat=miss_rgb=skip_used=0
    if not os.path.isfile(csv_path): raise FileNotFoundError(csv_path)
    csv_dir = os.path.dirname(os.path.abspath(csv_path))
    with open(csv_path, "r", encoding="utf-8") as f:
        rd = csv.DictReader(f)
        for row in rd:
            heat_path = (row.get("heat_png") or "").strip()
            img_fn    = (row.get("img_filename") or "").strip()
            h = _float_or_none(row.get("height_m", ""))
            if not heat_path or h is None: continue
            if (not os.path.isabs(heat_path)) or (not os.path.isfile(heat_path)):
                hp = _normalize_path(heat_path, csv_dir)
                heat_path = hp if (hp and os.path.isfile(hp)) else heat_path
            if not os.path.isfile(heat_path): miss_heat+=1; continue
            rgb_path = _find_rgb_path_from_filename(rgb_root, img_fn)
            if not rgb_path or not os.path.isfile(rgb_path): miss_rgb+=1; continue
            if filter_used_flag:
                uhw = str(row.get("used_height_weight", "1")).strip()
                try:
                    if int(float(uhw)) < 1: skip_used+=1; continue
                except: pass
            items.append({"heat_path":heat_path, "rgb_path":rgb_path, "height_m":float(h)})
    print(f"[MANIFEST] loaded={len(items)} from {os.path.basename(csv_path)} | miss_heat={miss_heat}, miss_rgb={miss_rgb}, skipped={skip_used}")
    return items

def _paired_random_resized_crop(rgb_t, heat_t, size_hw, scale, ratio):
    i, j, h, w = RandomResizedCrop.get_params(rgb_t, scale=scale, ratio=ratio)
    H, W = size_hw
    rgb_t  = TF.resized_crop(rgb_t,  i, j, h, w, size=(H, W), interpolation=InterpolationMode.BILINEAR, antialias=True)
    heat_t = TF.resized_crop(heat_t, i, j, h, w, size=(H, W), interpolation=InterpolationMode.BILINEAR, antialias=True)
    return rgb_t, heat_t

def _apply_color_jitter_rgb(rgb_t):
    if random.random() < AUG_COLORJIT_P:
        if AUG_BRIGHTNESS > 0:
            low = max(0.0, 1.0 - AUG_BRIGHTNESS); hi = 1.0 + AUG_BRIGHTNESS
            rgb_t = TF.adjust_brightness(rgb_t, random.uniform(low, hi))
        if AUG_CONTRAST > 0:
            low = max(0.0, 1.0 - AUG_CONTRAST); hi = 1.0 + AUG_CONTRAST
            rgb_t = TF.adjust_contrast(rgb_t, random.uniform(low, hi))
        if AUG_SATURATION > 0:
            low = max(0.0, 1.0 - AUG_SATURATION); hi = 1.0 + AUG_SATURATION
            rgb_t = TF.adjust_saturation(rgb_t, random.uniform(low, hi))
        if AUG_HUE > 0:
            hue = random.uniform(-AUG_HUE, AUG_HUE)
            rgb_t = TF.adjust_hue(rgb_t, hue)
    if random.random() < AUG_GAMMA_P:
        gamma = random.uniform(*AUG_GAMMA_RANGE)
        rgb_t = TF.adjust_gamma(torch.clamp(rgb_t, 0.0, 1.0), gamma=gamma, gain=1.0)
    if random.random() < AUG_GRAY_P:
        g = TF.rgb_to_grayscale(rgb_t, num_output_channels=1)
        rgb_t = g.repeat(3, 1, 1)
    if random.random() < AUG_GBLUR_P:
        k = int(AUG_GBLUR_K) if int(AUG_GBLUR_K) % 2 == 1 else int(AUG_GBLUR_K) + 1
        sigma = random.uniform(*AUG_GBLUR_SIGMA)
        rgb_t = TF.gaussian_blur(rgb_t, kernel_size=[k, k], sigma=sigma)
    return rgb_t

def apply_paired_augs_torchvision(rgb_t, heat_t):
    if not AUG_ENABLE:
        return rgb_t, heat_t
    if AUG_RANDOM_RESIZED_CROP:
        rgb_t, heat_t = _paired_random_resized_crop(rgb_t, heat_t, (IMG_SIZE, IMG_SIZE), AUG_RRC_SCALE, AUG_RRC_RATIO)
    if random.random() < AUG_HFLIP_P:
        rgb_t  = TF.hflip(rgb_t); heat_t = TF.hflip(heat_t)
    if random.random() < AUG_VFLIP_P:
        rgb_t  = TF.vflip(rgb_t); heat_t = TF.vflip(heat_t)
    if random.random() < AUG_TRANSPOSE90_P:
        rgb_t  = rgb_t.transpose(-1, -2).flip(-1)
        heat_t = heat_t.transpose(-1, -2).flip(-1)
    if AUG_ROTATE_DEG and AUG_ROTATE_DEG > 0:
        angle = random.uniform(-AUG_ROTATE_DEG, AUG_ROTATE_DEG)
        rgb_t  = TF.rotate(rgb_t,  angle, interpolation=InterpolationMode.BILINEAR, fill=[0.0, 0.0, 0.0])
        heat_t = TF.rotate(heat_t, angle, interpolation=InterpolationMode.BILINEAR, fill=[0.0])
    rgb_t = _apply_color_jitter_rgb(rgb_t)
    if (rgb_t.shape[-2], rgb_t.shape[-1]) != (IMG_SIZE, IMG_SIZE):
        rgb_t  = TF.resize(rgb_t,  [IMG_SIZE, IMG_SIZE], interpolation=InterpolationMode.BILINEAR, antialias=True)
    if (heat_t.shape[-2], heat_t.shape[-1]) != (IMG_SIZE, IMG_SIZE):
        heat_t = TF.resize(heat_t, [IMG_SIZE, IMG_SIZE], interpolation=InterpolationMode.BILINEAR, antialias=True)
    return rgb_t, heat_t

class HeatLateFusionDataset(Dataset):
    def __init__(self, records, augment=False, y_mu=0.0, y_std=1.0):
        self.recs = records; self.augment = augment
        self.rgb_mean  = np.array([0.485, 0.456, 0.406], dtype=np.float32)
        self.rgb_std   = np.array([0.229, 0.224, 0.225], dtype=np.float32)
        self.heat_mean = np.array([0.5], dtype=np.float32)
        self.heat_std  = np.array([0.25], dtype=np.float32)
        self.y_mu = float(y_mu); self.y_std = float(y_std)

    def __len__(self): return len(self.recs)

    def __getitem__(self, i):
        r = self.recs[i]
        rgb = cv2.imread(r["rgb_path"], cv2.IMREAD_COLOR)
        if rgb is None: raise FileNotFoundError(r["rgb_path"])
        rgb = cv2.cvtColor(rgb, cv2.COLOR_BGR2RGB).astype(np.float32)/255.0
        heat = cv2.imread(r["heat_path"], cv2.IMREAD_UNCHANGED)
        if heat is None: raise FileNotFoundError(r["heat_path"])
        if heat.ndim == 3: heat = cv2.cvtColor(heat, cv2.COLOR_BGR2GRAY)
        heat = heat.astype(np.float32);
        if heat.max() > 1.5: heat /= 255.0
        if not AUG_RANDOM_RESIZED_CROP:
            if (rgb.shape[0], rgb.shape[1]) != (IMG_SIZE, IMG_SIZE):
                rgb  = cv2.resize(rgb,  (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_LINEAR)
            if (heat.shape[0], heat.shape[1]) != (IMG_SIZE, IMG_SIZE):
                heat = cv2.resize(heat, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_LINEAR)
        rgb_t  = torch.from_numpy(rgb.transpose(2,0,1).copy()).float()
        heat_t = torch.from_numpy(heat[None, ...].copy()).float()
        if self.augment: rgb_t, heat_t = apply_paired_augs_torchvision(rgb_t, heat_t)
        else:
            if (rgb_t.shape[-2], rgb_t.shape[-1]) != (IMG_SIZE, IMG_SIZE):
                rgb_t  = TF.resize(rgb_t,  [IMG_SIZE, IMG_SIZE], interpolation=InterpolationMode.BILINEAR, antialias=True)
            if (heat_t.shape[-2], heat_t.shape[-1]) != (IMG_SIZE, IMG_SIZE):
                heat_t = TF.resize(heat_t, [IMG_SIZE, IMG_SIZE], interpolation=InterpolationMode.BILINEAR, antialias=True)
        rgb_mean_t  = torch.tensor(self.rgb_mean[:, None, None], dtype=rgb_t.dtype)
        rgb_std_t   = torch.tensor(self.rgb_std[:, None, None],  dtype=rgb_t.dtype)
        heat_mean_t = torch.tensor(self.heat_mean[:, None, None], dtype=heat_t.dtype)
        heat_std_t  = torch.tensor(self.heat_std[:, None, None],  dtype=heat_t.dtype)
        rgb_t  = (rgb_t  - rgb_mean_t)  / rgb_std_t
        heat_t = (heat_t - heat_mean_t) / heat_std_t
        y_scaled = (r["height_m"] - self.y_mu) / self.y_std
        return {"rgb":rgb_t.contiguous(), "heat":heat_t.contiguous(),
                "y_scaled":torch.tensor([y_scaled], dtype=torch.float32),
                "height_true_m":torch.tensor([r["height_m"]], dtype=torch.float32),
                "heat_path": r["heat_path"]}

def make_1ch_conv_from_3ch(conv3: nn.Conv2d) -> nn.Conv2d:
    new = nn.Conv2d(1, conv3.out_channels, kernel_size=conv3.kernel_size,
                    stride=conv3.stride, padding=conv3.padding, bias=False)
    with torch.no_grad():
        new.weight.copy_(conv3.weight.mean(dim=1, keepdim=True))
    return new

class LateFusionResNetRegressor(nn.Module):
    def __init__(
        self,
        fusion_hidden=512,
        dropout=0.0,
        out_dim=1,
        verbose=True,
        # 단일 모델과 동일하게 맞추기 위한 옵션(기본값 True)
        freeze_rgb_stem=True,
        freeze_heat_stem=True,
        freeze_layer1=False,
        remove_maxpool=False,
    ):
        super().__init__()
        rgb_enc, feat_dim = load_resnet(
            RGB_BACKEND, RGB_ARCH, RGB_WEIGHTS, remove_fc=True, verbose=verbose
        )
        heat_enc, _ = load_resnet(
            HEAT_BACKEND, HEAT_ARCH, HEAT_WEIGHTS, remove_fc=True, verbose=False
        )

        # 선택적으로 maxpool 제거(단일 코드와 동일 기본값 False)
        if remove_maxpool and hasattr(rgb_enc, "maxpool"):
            rgb_enc.maxpool = nn.Identity()
        if remove_maxpool and hasattr(heat_enc, "maxpool"):
            heat_enc.maxpool = nn.Identity()

        # Heat는 1ch 입력
        heat_enc.conv1 = make_1ch_conv_from_3ch(heat_enc.conv1)

        # stem 동결 옵션
        if freeze_rgb_stem:
            for name in ["conv1", "bn1"]:
                if hasattr(rgb_enc, name):
                    for p in getattr(rgb_enc, name).parameters():
                        p.requires_grad = False
        if freeze_heat_stem:
            for name in ["conv1", "bn1"]:
                if hasattr(heat_enc, name):
                    for p in getattr(heat_enc, name).parameters():
                        p.requires_grad = False
        if freeze_layer1:
            if hasattr(rgb_enc, "layer1"):
                for p in rgb_enc.layer1.parameters():
                    p.requires_grad = False
            if hasattr(heat_enc, "layer1"):
                for p in heat_enc.layer1.parameters():
                    p.requires_grad = False

        self.rgb_encoder = rgb_enc
        self.heat_encoder = heat_enc
        self.head = nn.Sequential(
            nn.Linear(feat_dim * 2, fusion_hidden),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),
            nn.Linear(fusion_hidden, out_dim),
        )

    def forward(self, x_rgb, x_heat):
        fr = self.rgb_encoder(x_rgb)
        fh = self.heat_encoder(x_heat)
        return self.head(torch.cat([fr, fh], dim=1))

def make_loaders(train_items, val_items, seed, y_mu, y_std):
    def seed_worker(worker_id):
        s = seed + worker_id
        random.seed(s); np.random.seed(s); torch.manual_seed(s)
    g = torch.Generator(); g.manual_seed(seed)
    train_ds = HeatLateFusionDataset(train_items, augment=True,  y_mu=y_mu, y_std=y_std)
    val_ds   = HeatLateFusionDataset(val_items,   augment=False, y_mu=y_mu, y_std=y_std)
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,
                              num_workers=NUM_WORKERS, pin_memory=True,
                              worker_init_fn=seed_worker, generator=g,
                              persistent_workers=(NUM_WORKERS>0))
    val_loader   = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False,
                              num_workers=NUM_WORKERS, pin_memory=True,
                              worker_init_fn=seed_worker, generator=g,
                              persistent_workers=(NUM_WORKERS>0))
    return train_loader, val_loader

@torch.no_grad()
def evaluate(m, loader, criterion, y_mu, y_std):
    m.eval(); rows=[]; loss_sum=0.0; n_total=0
    for b in loader:
        rgb  = b["rgb"].to(DEVICE)
        heat = b["heat"].to(DEVICE)
        y_s  = b["y_scaled"].to(DEVICE)
        pred_s = m(rgb, heat)
        loss_batch = criterion(pred_s, y_s)
        bs = int(rgb.size(0)); loss_sum += float(loss_batch.item())*bs; n_total += bs
        yt = (y_s.cpu().numpy().reshape(-1) * y_std) + y_mu
        yp = (pred_s.cpu().numpy().reshape(-1) * y_std) + y_mu
        heat_paths = b["heat_path"]
        for i in range(len(yt)):
            ae = float(abs(yp[i]-yt[i])); rows.append((heat_paths[i], float(yt[i]), float(yp[i]), ae))
    rmse = float(np.sqrt(np.mean([r[3]**2 for r in rows]))) if rows else 0.0
    val_loss = loss_sum / max(1, n_total)
    return val_loss, rmse, rows

def _save_curve(xs, ys_list, labels, ylabel, out_png):
    plt.figure(figsize=(7,4.2))
    for ys, lb in zip(ys_list, labels):
        plt.plot(xs, ys, marker="o", linewidth=2, label=lb)
    plt.xlabel("Epoch"); plt.ylabel(ylabel); plt.grid(True, alpha=0.3); plt.legend()
    plt.tight_layout(); plt.savefig(out_png, dpi=220); plt.close()
    print(f"[저장] {out_png}")

def _save_scatter(rows, title, out_png):
    gt = np.array([r[1] for r in rows], dtype=float)
    pd = np.array([r[2] for r in rows], dtype=float)
    min_v = float(min(gt.min(), pd.min())); max_v = float(max(gt.max(), pd.max()))
    pad = 0.02 * (max_v - min_v + 1e-6); xmin, xmax = min_v - pad, max_v + pad
    plt.figure(figsize=(5.4,5.4))
    plt.scatter(gt, pd, s=12, alpha=0.6, edgecolors="none")
    plt.plot([xmin, xmax], [xmin, xmax], "r--", linewidth=2, label="y = x")
    plt.xlim(xmin, xmax); plt.ylim(xmin, xmax); plt.xlabel("True Height (m)"); plt.ylabel("Predicted Height (m)")
    plt.title(title); plt.legend(); plt.tight_layout(); plt.savefig(out_png, dpi=220); plt.close()
    print(f"[저장] {out_png}")

def _prepare_data_and_stats():
    set_global_seed(BASE_SEED)
    train_items = load_items_from_manifest_csv(TRAIN_CSV_MANIFEST, TRAIN_RGB_ROOT, filter_used_flag=True)
    val_items   = load_items_from_manifest_csv(VAL_CSV_MANIFEST,   VAL_RGB_ROOT,   filter_used_flag=False)
    if len(train_items)==0 or len(val_items)==0:
        raise SystemExit(f"[ERROR] 샘플 부족: train={len(train_items)}, val={len(val_items)}")
    y_train = np.array([it["height_m"] for it in train_items], dtype=np.float32)
    y_mu = float(y_train.mean()); y_std = float(y_train.std()) if y_train.std() > 1e-6 else 1.0
    print(f"[INFO] Train={len(train_items)}, Val={len(val_items)}, y_mu={y_mu:.3f}, y_std={y_std:.3f}")
    return train_items, val_items, y_mu, y_std

def train_member(member_id:int, epochs:int=EPOCHS):
    member_dir = os.path.join(OUT_BASE, f"member_{member_id:02d}")
    os.makedirs(member_dir, exist_ok=True)
    seed = BASE_SEED + member_id
    set_global_seed(seed)
    train_items, val_items, y_mu, y_std = _prepare_data_and_stats()
    train_loader, val_loader = make_loaders(train_items, val_items, seed, y_mu, y_std)

    model = LateFusionResNetRegressor(
    fusion_hidden=512,
    dropout=0.0,               # 단일 모델과 동일
    out_dim=1,
    verbose=(member_id==0),
    freeze_rgb_stem=True,      # ★ 단일 학습 코드와 동일
    freeze_heat_stem=True,     # ★ 단일 학습 코드와 동일
    freeze_layer1=False,
    remove_maxpool=False
    ).to(DEVICE)

    criterion = nn.MSELoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.5, patience=3)

    history = []; best_rmse, best_ep = float("inf"), -1
    best_path = os.path.join(member_dir, "best_rmse_latefusion_rgb_heat.pt")

    from tqdm import tqdm
    for ep in range(1, epochs+1):
        model.train(); tr_sum=0.0; tr_n=0
        for b in tqdm(train_loader, desc=f"[M{member_id}] Epoch {ep}/{epochs}"):
            rgb=b["rgb"].to(DEVICE); heat=b["heat"].to(DEVICE); y_s=b["y_scaled"].to(DEVICE)
            optimizer.zero_grad(set_to_none=True)
            pred = model(rgb, heat)
            loss = criterion(pred, y_s)
            loss.backward(); nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            bs = int(rgb.size(0)); tr_sum += float(loss.detach().item())*bs; tr_n += bs
        tr_loss = tr_sum / max(1, tr_n)
        val_loss, val_rmse, _ = evaluate(model, val_loader, criterion, y_mu, y_std)
        scheduler.step(val_rmse)
        history.append((ep, tr_loss, val_loss, val_rmse))
        print(f"[M{member_id}] ep{ep:02d} | train {tr_loss:.4f} | val {val_loss:.4f} | RMSE {val_rmse:.3f}")
        if val_rmse < best_rmse:
            best_rmse, best_ep = val_rmse, ep
            torch.save(model.state_dict(), best_path)

    # 기록 저장
    hist_csv = os.path.join(member_dir, "history.csv")
    with open(hist_csv, "w", newline="", encoding="utf-8") as f:
        wr = csv.writer(f); wr.writerow(["epoch","train_loss","val_loss","val_rmse"])
        for ep, tr, vl, vr in history: wr.writerow([ep, f"{tr:.6f}", f"{vl:.6f}", f"{vr:.6f}"])
    # 곡선 저장
    xs = [h[0] for h in history]
    _save_curve(xs, [[h[1] for h in history],[h[2] for h in history]], ["Train","Val"], "MSE (scaled)", os.path.join(member_dir, "loss_curve.png"))
    _save_curve(xs, [[h[3] for h in history]], ["Val RMSE"], "RMSE (m)", os.path.join(member_dir, "rmse_curve.png"))

    # best 로드 후 개별 성능/산점도
    best_model = LateFusionResNetRegressor(
    fusion_hidden=512, dropout=0.0, out_dim=1, verbose=False,
    freeze_rgb_stem=True, freeze_heat_stem=True, freeze_layer1=False, remove_maxpool=False
    ).to(DEVICE)

    best_model.load_state_dict(torch.load(best_path, map_location=DEVICE))
    _, val_rmse, val_rows = evaluate(best_model, val_loader, criterion, y_mu, y_std)

    pred_csv = os.path.join(member_dir, "val_predictions_member.csv")
    with open(pred_csv, "w", newline="", encoding="utf-8") as f:
        wr = csv.writer(f); wr.writerow(["heat_path","height_true_m","height_pred_m","abs_err_m"])
        for hp, yt, yp, ae in val_rows: wr.writerow([hp, f"{yt:.6f}", f"{yp:.6f}", f"{ae:.6f}"])
    _save_scatter(val_rows, f"[Member {member_id}] Validation Scatter (RMSE={val_rmse:.2f} m)",
                  os.path.join(member_dir, "scatter_val_true_vs_pred.png"))
    # 요약 저장
    with open(os.path.join(member_dir, "summary.json"), "w", encoding="utf-8") as f:
        json.dump({"member_id":member_id,"best_ep":best_ep,"val_rmse":val_rmse,"best_path":best_path}, f, ensure_ascii=False, indent=2)
    print(f"[DONE] Member {member_id} 완료, best_ep={best_ep}, RMSE={val_rmse:.3f}")
    return {"member_id":member_id, "best_ep":best_ep, "val_rmse":val_rmse, "best_path":best_path}

@torch.no_grad()
def _evaluate_ensemble(members, loader, y_mu, y_std):
    criterion = nn.MSELoss()
    rows=[]; loss_sum=0.0; n_total=0
    for b in loader:
        rgb=b["rgb"].to(DEVICE); heat=b["heat"].to(DEVICE); y_s=b["y_scaled"].to(DEVICE)
        preds = [m(rgb, heat) for m in members]
        pred_s = torch.stack(preds, dim=0).mean(dim=0)
        loss_batch = criterion(pred_s, y_s)
        bs = int(rgb.size(0)); loss_sum += float(loss_batch.item())*bs; n_total += bs
        yt = (y_s.cpu().numpy().reshape(-1) * y_std) + y_mu
        yp = (pred_s.cpu().numpy().reshape(-1) * y_std) + y_mu
        heat_paths = b["heat_path"]
        for i in range(len(yt)):
            ae = float(abs(yp[i]-yt[i])); rows.append((heat_paths[i], float(yt[i]), float(yp[i]), ae))
    rmse = float(np.sqrt(np.mean([r[3]**2 for r in rows]))) if rows else 0.0
    val_loss = loss_sum / max(1, n_total)
    return val_loss, rmse, rows

def try_ensemble_if_ready():
    # 모든 멤버 가중치가 존재하면 앙상블을 수행한다.
    missing = []
    best_paths = []
    for i in range(ENSEMBLE_SIZE):
        p = os.path.join(OUT_BASE, f"member_{i:02d}", "best_rmse_latefusion_rgb_heat.pt")
        if not os.path.isfile(p): missing.append(i)
        else: best_paths.append(p)
    if missing:
        print(f"[ENSEMBLE] 대기 중: 아직 학습 완료되지 않은 멤버 = {missing}")
        return None

    print("[ENSEMBLE] 모든 멤버 가중치 확인, 앙상블 평가 시작")
    # 데이터/로더 재구성
    set_global_seed(BASE_SEED + 777)
    train_items = load_items_from_manifest_csv(TRAIN_CSV_MANIFEST, TRAIN_RGB_ROOT, filter_used_flag=True)
    val_items   = load_items_from_manifest_csv(VAL_CSV_MANIFEST,   VAL_RGB_ROOT,   filter_used_flag=False)
    y_train = np.array([it["height_m"] for it in train_items], dtype=np.float32)
    y_mu = float(y_train.mean()); y_std = float(y_train.std()) if y_train.std() > 1e-6 else 1.0
    def seed_worker(worker_id):
        s = (BASE_SEED+777) + worker_id
        random.seed(s); np.random.seed(s); torch.manual_seed(s)
    g = torch.Generator(); g.manual_seed(BASE_SEED+777)
    val_ds = HeatLateFusionDataset(val_items, augment=False, y_mu=y_mu, y_std=y_std)
    val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False,
                            num_workers=NUM_WORKERS, pin_memory=True,
                            worker_init_fn=seed_worker, generator=g,
                            persistent_workers=(NUM_WORKERS>0))
    # 멤버 로드
    members=[]
    for p in best_paths:
        m = LateFusionResNetRegressor(fusion_hidden=512, dropout=0.0, out_dim=1, verbose=False).to(DEVICE)
        m.load_state_dict(torch.load(p, map_location=DEVICE)); m.eval(); members.append(m)
    ens_val_loss, ens_val_rmse, ens_rows = _evaluate_ensemble(members, val_loader, y_mu, y_std)
    # 저장
    pred_csv = os.path.join(OUT_BASE, "val_predictions_ensemble.csv")
    with open(pred_csv, "w", newline="", encoding="utf-8") as f:
        wr = csv.writer(f); wr.writerow(["heat_path","height_true_m","height_pred_m","abs_err_m"])
        for hp, yt, yp, ae in ens_rows: wr.writerow([hp, f"{yt:.6f}", f"{yp:.6f}", f"{ae:.6f}"])
    _save_scatter(ens_rows, f"[Ensemble x{ENSEMBLE_SIZE}] Validation Scatter (RMSE={ens_val_rmse:.2f} m)",
                  os.path.join(OUT_BASE, "ensemble_scatter_val_true_vs_pred.png"))
    with open(os.path.join(OUT_BASE, "ensemble_metrics.txt"), "w") as f:
        f.write(f"Members: {ENSEMBLE_SIZE}\n")
        for i,p in enumerate(best_paths): f.write(f"- M{i:02d}: {p}\n")
        f.write(f"Ensemble Val Loss (scaled MSE): {ens_val_loss:.6f}\n")
        f.write(f"Ensemble Val RMSE (m): {ens_val_rmse:.6f}\n")
    print(f"[ENSEMBLE DONE] RMSE={ens_val_rmse:.4f}, csv={pred_csv}")

    # === 멤버 종합 곡선 저장 ===
    plot_combined_member_curves(out_base=OUT_BASE, ensemble_size=ENSEMBLE_SIZE)
    return {"rmse":ens_val_rmse, "csv":pred_csv}

def _load_history_csv(csv_path):
    ep, tr, vl, vr = [], [], [], []
    if not os.path.isfile(csv_path): return ep, tr, vl, vr
    with open(csv_path, "r", encoding="utf-8") as f:
        rd = csv.DictReader(f)
        for row in rd:
            try:
                ep.append(int(row["epoch"]))
                tr.append(float(row["train_loss"]))
                vl.append(float(row["val_loss"]))
                vr.append(float(row["val_rmse"]))
            except: pass
    return ep, tr, vl, vr

def plot_combined_member_curves(out_base=OUT_BASE, ensemble_size=ENSEMBLE_SIZE):
    mdirs = [os.path.join(out_base, f"member_{i:02d}") for i in range(ensemble_size)]
    def _plot(metric_key, out_png, ylabel):
        import matplotlib.pyplot as plt
        plt.figure(figsize=(8,4.5)); any_data=False
        for i, mdir in enumerate(mdirs):
            hist = os.path.join(mdir, "history.csv")
            ep, tr, vl, vr = _load_history_csv(hist)
            if not ep: continue
            any_data=True
            ys = tr if metric_key=="train" else (vl if metric_key=="val" else vr)
            plt.plot(ep, ys, marker="o", linewidth=2, label=f"M{i:02d}")
        if not any_data:
            print(f"[WARN] 멤버 history 없음: {metric_key}"); plt.close(); return
        plt.xlabel("Epoch"); plt.ylabel(ylabel); plt.grid(True, alpha=0.3); plt.legend(ncol=3, fontsize=9)
        plt.tight_layout(); plt.savefig(out_png, dpi=240); plt.close(); print(f"[저장] {out_png}")
    _plot("train", os.path.join(out_base, "combined_train_loss_all_members.png"), "Train MSE (scaled)")
    _plot("val",   os.path.join(out_base, "combined_val_loss_all_members.png"),   "Val MSE (scaled)")
    _plot("rmse",  os.path.join(out_base, "combined_val_rmse_all_members.png"),   "Val RMSE (m)")

print("[READY] 공통 모듈 로딩/정의 완료")


[READY] 공통 모듈 로딩/정의 완료


1. Member 0 학습 진행

In [ ]:
# ===== Member 0 =====
from __main__ import train_member, try_ensemble_if_ready
info0 = train_member(member_id=0)
ens0 = try_ensemble_if_ready()

[MANIFEST] loaded=17250 from train_manifest_regions_uniform.csv | miss_heat=0, miss_rgb=0, skipped=0
[MANIFEST] loaded=1323 from val_manifest_regions.csv | miss_heat=0, miss_rgb=0, skipped=0
[INFO] Train=17250, Val=1323, y_mu=159.844, y_std=81.022
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 222MB/s]


[load] backend=torchvision arch=resnet18 weights=IMAGENET1K_V1 -> feat_dim=512


[M0] Epoch 1/50:   0%|          | 0/270 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torch/nn/modules/linear.py:125: UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`, but this operation is not deterministic because it uses CuBLAS and you have CUDA >= 10.2. To enable deterministic behavior in this case, you must set an environment variable before running your PyTorch application: CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CONFIG=:16:8. For more information, go to https://docs.nvidia.com/cuda/cublas/index.html#results-reproducibility (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:233.)
  return F.linear(input, self.weight, self.bias)
/usr/local/lib/python3.12/dist-packages/torch/autograd/graph.py:829: UserWarning: Deterministic behavior was enabled with either `torch.use_deterministic_algorithms(True)` or `at::Context::setDeterministicAlgorithms(true)`

[M0] ep01 | train 0.1260 | val 0.0465 | RMSE 17.468


...

[M0] ep48 | train 0.0006 | val 0.0033 | RMSE 4.670


[M0] Epoch 49/50: 100%|██████████| 270/270 [02:41<00:00,  1.67it/s]


[M0] ep49 | train 0.0007 | val 0.0033 | RMSE 4.658


[M0] Epoch 50/50: 100%|██████████| 270/270 [02:41<00:00,  1.68it/s]


[M0] ep50 | train 0.0006 | val 0.0032 | RMSE 4.563
[저장] /content/drive/MyDrive/DCC2025/Mission_2/latefusion_ensemble/member_00/loss_curve.png
[저장] /content/drive/MyDrive/DCC2025/Mission_2/latefusion_ensemble/member_00/rmse_curve.png
[저장] /content/drive/MyDrive/DCC2025/Mission_2/latefusion_ensemble/member_00/scatter_val_true_vs_pred.png
[DONE] Member 0 완료, best_ep=45, RMSE=4.544
[ENSEMBLE] 모든 멤버 가중치 확인, 앙상블 평가 시작
[MANIFEST] loaded=17250 from train_manifest_regions_uniform.csv | miss_heat=0, miss_rgb=0, skipped=0
[MANIFEST] loaded=1323 from val_manifest_regions.csv | miss_heat=0, miss_rgb=0, skipped=0
[저장] /content/drive/MyDrive/DCC2025/Mission_2/latefusion_ensemble/ensemble_scatter_val_true_vs_pred.png
[ENSEMBLE DONE] RMSE=4.3887, csv=/content/drive/MyDrive/DCC2025/Mission_2/latefusion_ensemble/val_predictions_ensemble.csv
[저장] /content/drive/MyDrive/DCC2025/Mission_2/latefusion_ensemble/combined_train_loss_all_members.png
[저장] /content/drive/MyDrive/DCC2025/Mission_2/latefusion_ensembl

2. Member 1 학습 진행

In [ ]:
# ===== Member 1 =====
from __main__ import train_member, try_ensemble_if_ready
info1 = train_member(member_id=1)
ens1 = try_ensemble_if_ready()


[MANIFEST] loaded=17250 from train_manifest_regions_uniform.csv | miss_heat=0, miss_rgb=0, skipped=0
[MANIFEST] loaded=1323 from val_manifest_regions.csv | miss_heat=0, miss_rgb=0, skipped=0
[INFO] Train=17250, Val=1323, y_mu=159.844, y_std=81.022


[M1] Epoch 1/50: 100%|██████████| 270/270 [02:41<00:00,  1.67it/s]


[M1] ep01 | train 0.1233 | val 0.0468 | RMSE 17.520


[M1] Epoch 2/50: 100%|██████████| 270/270 [02:41<00:00,  1.68it/s]


[M1] ep02 | train 0.0453 | val 0.0478 | RMSE 17.719


...

[M1] ep48 | train 0.0007 | val 0.0040 | RMSE 5.110


[M1] Epoch 49/50: 100%|██████████| 270/270 [02:41<00:00,  1.68it/s]


[M1] ep49 | train 0.0007 | val 0.0041 | RMSE 5.171


[M1] Epoch 50/50: 100%|██████████| 270/270 [02:41<00:00,  1.67it/s]


[M1] ep50 | train 0.0007 | val 0.0039 | RMSE 5.063
[저장] /content/drive/MyDrive/DCC2025/Mission_2/latefusion_ensemble/member_01/loss_curve.png
[저장] /content/drive/MyDrive/DCC2025/Mission_2/latefusion_ensemble/member_01/rmse_curve.png
[저장] /content/drive/MyDrive/DCC2025/Mission_2/latefusion_ensemble/member_01/scatter_val_true_vs_pred.png
[DONE] Member 1 완료, best_ep=46, RMSE=5.059
[ENSEMBLE] 모든 멤버 가중치 확인, 앙상블 평가 시작
[MANIFEST] loaded=17250 from train_manifest_regions_uniform.csv | miss_heat=0, miss_rgb=0, skipped=0
[MANIFEST] loaded=1323 from val_manifest_regions.csv | miss_heat=0, miss_rgb=0, skipped=0
[저장] /content/drive/MyDrive/DCC2025/Mission_2/latefusion_ensemble/ensemble_scatter_val_true_vs_pred.png
[ENSEMBLE DONE] RMSE=4.4154, csv=/content/drive/MyDrive/DCC2025/Mission_2/latefusion_ensemble/val_predictions_ensemble.csv
[저장] /content/drive/MyDrive/DCC2025/Mission_2/latefusion_ensemble/combined_train_loss_all_members.png
[저장] /content/drive/MyDrive/DCC2025/Mission_2/latefusion_ensembl

3. Member 2 학습 진행

In [ ]:
# ===== Member 2 =====
from __main__ import train_member, try_ensemble_if_ready
info2 = train_member(member_id=2)
ens2 = try_ensemble_if_ready()

[MANIFEST] loaded=17250 from train_manifest_regions_uniform.csv | miss_heat=0, miss_rgb=0, skipped=0
[MANIFEST] loaded=1323 from val_manifest_regions.csv | miss_heat=0, miss_rgb=0, skipped=0
[INFO] Train=17250, Val=1323, y_mu=159.844, y_std=81.022


[M2] Epoch 1/50: 100%|██████████| 270/270 [02:41<00:00,  1.67it/s]


[M2] ep01 | train 0.1399 | val 0.0457 | RMSE 17.317


[M2] Epoch 2/50: 100%|██████████| 270/270 [02:41<00:00,  1.68it/s]


[M2] ep02 | train 0.0412 | val 0.0249 | RMSE 12.774


...

[M2] ep48 | train 0.0006 | val 0.0036 | RMSE 4.874


[M2] Epoch 49/50: 100%|██████████| 270/270 [02:41<00:00,  1.68it/s]


[M2] ep49 | train 0.0006 | val 0.0037 | RMSE 4.926


[M2] Epoch 50/50: 100%|██████████| 270/270 [02:41<00:00,  1.68it/s]


[M2] ep50 | train 0.0006 | val 0.0035 | RMSE 4.777
[저장] /content/drive/MyDrive/DCC2025/Mission_2/latefusion_ensemble/member_02/loss_curve.png
[저장] /content/drive/MyDrive/DCC2025/Mission_2/latefusion_ensemble/member_02/rmse_curve.png
[저장] /content/drive/MyDrive/DCC2025/Mission_2/latefusion_ensemble/member_02/scatter_val_true_vs_pred.png
[DONE] Member 2 완료, best_ep=50, RMSE=4.777
[ENSEMBLE] 모든 멤버 가중치 확인, 앙상블 평가 시작
[MANIFEST] loaded=17250 from train_manifest_regions_uniform.csv | miss_heat=0, miss_rgb=0, skipped=0
[MANIFEST] loaded=1323 from val_manifest_regions.csv | miss_heat=0, miss_rgb=0, skipped=0
[저장] /content/drive/MyDrive/DCC2025/Mission_2/latefusion_ensemble/ensemble_scatter_val_true_vs_pred.png
[ENSEMBLE DONE] RMSE=4.4449, csv=/content/drive/MyDrive/DCC2025/Mission_2/latefusion_ensemble/val_predictions_ensemble.csv
[저장] /content/drive/MyDrive/DCC2025/Mission_2/latefusion_ensemble/combined_train_loss_all_members.png
[저장] /content/drive/MyDrive/DCC2025/Mission_2/latefusion_ensembl

4. Member 3 학습 진행

In [ ]:
# ===== Member 3 =====
from __main__ import train_member, try_ensemble_if_ready
info3 = train_member(member_id=3)
ens3 = try_ensemble_if_ready()


[MANIFEST] loaded=17250 from train_manifest_regions_uniform.csv | miss_heat=0, miss_rgb=0, skipped=0
[MANIFEST] loaded=1323 from val_manifest_regions.csv | miss_heat=0, miss_rgb=0, skipped=0
[INFO] Train=17250, Val=1323, y_mu=159.844, y_std=81.022


[M3] Epoch 1/50: 100%|██████████| 270/270 [02:41<00:00,  1.67it/s]


[M3] ep01 | train 0.1159 | val 0.0429 | RMSE 16.778


[M3] Epoch 2/50: 100%|██████████| 270/270 [02:41<00:00,  1.67it/s]


[M3] ep02 | train 0.0485 | val 0.0343 | RMSE 15.015


...

[M3] ep48 | train 0.0006 | val 0.0035 | RMSE 4.777


[M3] Epoch 49/50: 100%|██████████| 270/270 [02:41<00:00,  1.67it/s]


[M3] ep49 | train 0.0006 | val 0.0035 | RMSE 4.790


[M3] Epoch 50/50: 100%|██████████| 270/270 [02:41<00:00,  1.67it/s]


[M3] ep50 | train 0.0006 | val 0.0035 | RMSE 4.767
[저장] /content/drive/MyDrive/DCC2025/Mission_2/latefusion_ensemble/member_03/loss_curve.png
[저장] /content/drive/MyDrive/DCC2025/Mission_2/latefusion_ensemble/member_03/rmse_curve.png
[저장] /content/drive/MyDrive/DCC2025/Mission_2/latefusion_ensemble/member_03/scatter_val_true_vs_pred.png
[DONE] Member 3 완료, best_ep=50, RMSE=4.767
[ENSEMBLE] 모든 멤버 가중치 확인, 앙상블 평가 시작
[MANIFEST] loaded=17250 from train_manifest_regions_uniform.csv | miss_heat=0, miss_rgb=0, skipped=0
[MANIFEST] loaded=1323 from val_manifest_regions.csv | miss_heat=0, miss_rgb=0, skipped=0
[저장] /content/drive/MyDrive/DCC2025/Mission_2/latefusion_ensemble/ensemble_scatter_val_true_vs_pred.png
[ENSEMBLE DONE] RMSE=4.4673, csv=/content/drive/MyDrive/DCC2025/Mission_2/latefusion_ensemble/val_predictions_ensemble.csv
[저장] /content/drive/MyDrive/DCC2025/Mission_2/latefusion_ensemble/combined_train_loss_all_members.png
[저장] /content/drive/MyDrive/DCC2025/Mission_2/latefusion_ensembl

5. Member 4 학습 진행 및 Ensemble 수행

In [ ]:
# ===== Member 4 =====
from __main__ import train_member, try_ensemble_if_ready
info4 = train_member(member_id=4)
ens_final = try_ensemble_if_ready()  # 다섯 멤버 가중치가 모두 존재하면 여기서 앙상블을 수행한다

[MANIFEST] loaded=17250 from train_manifest_regions_uniform.csv | miss_heat=0, miss_rgb=0, skipped=0
[MANIFEST] loaded=1323 from val_manifest_regions.csv | miss_heat=0, miss_rgb=0, skipped=0
[INFO] Train=17250, Val=1323, y_mu=159.844, y_std=81.022


[M4] Epoch 1/50: 100%|██████████| 270/270 [02:41<00:00,  1.67it/s]


[M4] ep01 | train 0.1313 | val 0.0548 | RMSE 18.966


[M4] Epoch 2/50: 100%|██████████| 270/270 [02:41<00:00,  1.67it/s]


[M4] ep02 | train 0.0402 | val 0.0343 | RMSE 15.011


...

[M4] ep48 | train 0.0008 | val 0.0037 | RMSE 4.927


[M4] Epoch 49/50: 100%|██████████| 270/270 [02:41<00:00,  1.67it/s]


[M4] ep49 | train 0.0008 | val 0.0036 | RMSE 4.835


[M4] Epoch 50/50: 100%|██████████| 270/270 [02:41<00:00,  1.67it/s]


[M4] ep50 | train 0.0006 | val 0.0036 | RMSE 4.830
[저장] /content/drive/MyDrive/DCC2025/Mission_2/latefusion_ensemble/member_04/loss_curve.png
[저장] /content/drive/MyDrive/DCC2025/Mission_2/latefusion_ensemble/member_04/rmse_curve.png
[저장] /content/drive/MyDrive/DCC2025/Mission_2/latefusion_ensemble/member_04/scatter_val_true_vs_pred.png
[DONE] Member 4 완료, best_ep=45, RMSE=4.775
[ENSEMBLE] 모든 멤버 가중치 확인, 앙상블 평가 시작
[MANIFEST] loaded=17250 from train_manifest_regions_uniform.csv | miss_heat=0, miss_rgb=0, skipped=0
[MANIFEST] loaded=1323 from val_manifest_regions.csv | miss_heat=0, miss_rgb=0, skipped=0
[저장] /content/drive/MyDrive/DCC2025/Mission_2/latefusion_ensemble/ensemble_scatter_val_true_vs_pred.png
[ENSEMBLE DONE] RMSE=4.4457, csv=/content/drive/MyDrive/DCC2025/Mission_2/latefusion_ensemble/val_predictions_ensemble.csv
[저장] /content/drive/MyDrive/DCC2025/Mission_2/latefusion_ensemble/combined_train_loss_all_members.png
[저장] /content/drive/MyDrive/DCC2025/Mission_2/latefusion_ensembl

Ensemble Learning 결과 시각화

In [ ]:
from __main__ import plot_combined_member_curves
plot_combined_member_curves()

[저장] /content/drive/MyDrive/DCC2025/Mission_2/latefusion_ensemble/combined_train_loss_all_members.png
[저장] /content/drive/MyDrive/DCC2025/Mission_2/latefusion_ensemble/combined_val_loss_all_members.png
[저장] /content/drive/MyDrive/DCC2025/Mission_2/latefusion_ensemble/combined_val_rmse_all_members.png
